# Assignment 3
The data you will be using is the FOIA data gathered from the city of Ann Arbor on parking tickets. The repository is made up of a set of Excel files which you're going to have to figure out how to load into a pandas `DataFrame` (time to check the APIs!). The files may have more than one sheet in them (who knows why!?). I'd like you to answer the following questions for me.

This is a substantial amount of data. Therefore, things will take a while to run. For testing purposes, I would recommend using a reasonable representative subset before applying your functions on the entire dataframe. Make sure you remove any extraneous outputs before you turn in your final copy though!

## Question 1: load_ticket_data()
First, write the code to create a single `DataFrame` object in a function called `load_ticket_data()`. This function should return the full dataframe and take no parameters (you can assume the ticket files are in the same directory as your assignment notebook). Column labels should be as follows:

`['Ticket #', 'Badge', 'Issue Date ', 'IssueTime', 'Plate', 'State',
       'Make', 'Model', 'Violation', ' Description', 'Location', 'Meter',
       ' Fine ', 'Penalty']`


Here are some hints.
<ol>
<li> Be sure to scroll through every single sheet to make sure what rows should be dropped.</li>
<li> Make sure to exclude unnecessary footers and headers from the datafile.</li>
<li> Check if your header labels are correct.</li>
</ol>

In [1]:
def load_ticket_data():
    import xlrd
    import pandas as pd
    import numpy as np
    import re

    # Filter all warnings. If you would like to see the warnings, please comment the two lines below.
    import warnings
    warnings.filterwarnings('ignore')

    # YOUR CODE HERE
    
    # load all the Excel files in order to count the number of sheets in them
    file1 = xlrd.open_workbook('AnnArbor-TicketViolation2015.xls', on_demand=True)
    file2 = xlrd.open_workbook('AnnArbor-TicketViolation2016.xls', on_demand=True)
    file3 = xlrd.open_workbook('AnnArbor-TicketViolation2017.xls', on_demand=True)
    file4 = xlrd.open_workbook('AnnArbor-TicketViolation2018.xls', on_demand=True)
    file5 = xlrd.open_workbook('AnnArbor-TicketViolation2019.xls', on_demand=True)
    file6 = xlrd.open_workbook('AnnArbor-TicketViolation-jan2020.xls', on_demand=True)

    # create a list of Excel files
    files=[file1, file2, file3, file4, file5, file6]

    # create an empty dictionary to store resulting dataframes, and a file count accumulator for dictionary keys
    df_dict={}
    file_num=0

    # loop through the Excel files
    for i in files:
        file_num+=1

        # loop through the sheets in the Excel files
        for j in range(len(i.sheet_names())):

            # make the second row the header if we're looking at the first sheet of all Excel files except the last one
            if j==0 and file_num!=6:
                df=pd.read_excel(i, sheet_name=j, header=1)

            # make the second row the header and skip the last row if we're looking at the first sheet of the last Excel file
            elif j==0 and file_num==6:
                df=pd.read_excel(i, sheet_name=j, header=1, skipfooter=1)

            # skip the last row if we're looking at the last sheet of all Excel files and add a header
            elif j==(len(i.sheet_names())-1):
                df=pd.read_excel(i, sheet_name=j, header=None,
                                 names=['Ticket #','Badge','Issue Date ','IssueTime','Plate','State','Make','Model','Violation',' Description',
                                        'Location','Meter',' Fine ','Penalty'], skipfooter=1)

            # add a header for all other sheets of all Excel files
            else:
                df=pd.read_excel(i, sheet_name=j, header=None,
                                 names=['Ticket #','Badge','Issue Date ','IssueTime','Plate','State','Make','Model','Violation',' Description',
                                        'Location','Meter',' Fine ','Penalty'])

            # drop rows containing NaN values in the Badge column (these rows are blank for other columns too)
            df.dropna(subset=["Badge"], axis=0, inplace=True)

            # append the dataframe to the dictionary, resulting in a dictionary of dataframes
            df_dict[str(file_num)+"-"+str(j)]=df

    # create an empty dataframe
    full_df=pd.DataFrame()

    # loop through all the keys in the dictionary of dataframes
    for k in df_dict.keys():

        # concatenate all the dataframes within the dictionary to form one big dataframe
        full_df=pd.concat([full_df, df_dict[k]])

    return full_df
#     raise NotImplementedError()

In [2]:
import xlrd
import pandas as pd
import numpy as np
import re

df_1_test = load_ticket_data()
assert isinstance(df_1_test, pd.DataFrame), "Q1: What your function returns must be pd.DataFrame."
assert len(df_1_test) == 811439, "Q1: There should be 811439 rows in the dataframe."
assert len(df_1_test.columns) == 14, "Q1: There should be 14 columns in the dataframe."

## Question 2: generate_descriptors()
Write a function called `generate_descriptors(df)` which takes in the DataFrame you loaded from Question1 and returns a dataframe of all unique ticket descriptions and how frequent they are (e.g. it will tell you how many "HANDICAP" or "NO PERMITS U/M" tickets have been issued) for each of the following three time periods: morning (3 am to 11:59 am), afternoon (12 pm - 5:59 pm), and evening (6 pm - 2:59 am).

* Make sure you drop na values of input `df`. 
* The DataFrame which `generate_descriptors(df)` returns should have 3 rows and 51 columns. 
* Index should be labelled as `Morning`, `Afternoon`, and `Evening`. 
* Column names should be unique values of `df["Description"]`.   

In [3]:
def generate_descriptors(df):
    import xlrd
    import pandas as pd
    import numpy as np
    import re

    # Filter all warnings. If you would like to see the warnings, please comment the two lines below.
    import warnings
    warnings.filterwarnings('ignore')
    
    # YOUR CODE HERE
    
    # drop rows with NaN values in Description column
    df.dropna(subset=[" Description"], axis=0, inplace=True)

    # define a function to get time period based on time in IssueTime column
    def time_period(time):
    
        if time>=300 and time<=1159:
            return "Morning"
        elif time>=1200 and time<=1759:
            return "Afternoon"
        else:
            return "Evening"
    
    # create a new column TimePeriod specifying the time period based on the IssueTime column
    df["TimePeriod"]=df.apply(lambda x: time_period(x["IssueTime"]), axis=1)
    
#     # convert IssueTime column data type to string
#     df["IssueTime"]=df["IssueTime"].astype("str")
    
#     # remove decimal point and zeros from the time string
#     df["IssueTime"]=df["IssueTime"].str.replace(".0", "")
    
#     # ensure that time string has 4 digits by adding leading zeros where necessary
#     df['IssueTime']=df['IssueTime'].apply(lambda x: x.zfill(4))
    
#     # convert time string to datetime data type and format
#     df["IssueTime"]=pd.to_datetime(df["IssueTime"], format="%H%M")
    
#     # reset index to get unique indices
#     df.reset_index(inplace=True)

#     # create a new column with details of time periods based on the hour in IssueTime column
#     df["TimePeriod"]=pd.cut(df["IssueTime"].dt.hour, bins=[0, 3, 12, 18, 24], labels=["Evening", "Morning", "Afternoon", "Evening"], right=False, include_lowest=True, ordered=False)
    
    # strip extra white spaces from column names
    cols=list(df.columns)
    cols=[x.strip() for x in cols]
    df.columns=cols
    
    # group by TimePeriod and Description, aggregate with count of Description
    df_q2=df.groupby(["TimePeriod", "Description"])["Description"].count().unstack()
    
#     raise NotImplementedError()
    return df_q2

In [4]:
df_q2 = generate_descriptors(load_ticket_data())
assert df_q2.shape == (3, 51), "Q2: The shape of the DataFrame is incorrect."
assert "Morning" in df_q2.index, 'Q2: "Morning" shoud be in the index of the DataFrame'
assert "Afternoon" in df_q2.index, 'Q2: "Afternoon" should be in the index of the DataFrame'
assert "Evening" in df_q2.index, 'Q2: "Evening" should be in the index of the DataFrame'

## Question 3: common_car_make()
What is the most common make of car which received tickets from the state of NY? The answer should be a string.

In [5]:
def common_car_make():
    import xlrd
    import pandas as pd
    import numpy as np
    import re

    # Filter all warnings. If you would like to see the warnings, please comment the two lines below.
    import warnings
    warnings.filterwarnings('ignore')

    df = load_ticket_data()
    # YOUR CODE HERE
    
    # grouping by state and make of car, and then selecting the top-most sorted value for the state of NY gives the necessary answer
    return df.groupby(["State", "Make"])["Make"].count().unstack().loc['NY'].sort_values(ascending=False).index[0]
    
    # raise NotImplementedError()

In [6]:
answer3 = common_car_make()
assert isinstance(answer3, str), "Q3: Your answer should be a string type."

## Question 4: fine_per_plates()
Starting in 2004 Michigan moved to issuing plates with the format of ABC1234. That got me thinking, how many vanity plate holders there are in our dataset? Count for me the number of Michigan vehicles with plates in the following formats that have received a ticket:
- ABC1234
- ABC123
- 123ABC
- Vanity Plates (i.e. anything other than the aforementioned formats)

Complete the function `fine_per_plates()` returning a dictionary. The dictinary should be formatted as follows:
```
plates_dict = {"ABC1234":the_number_of_vehicles,
                "ABC123":the_number_of_vehicles,
                "123ABC":the_number_of_vehicles,
                "vanity":the_number_of_vehicles}
```

In [7]:
def fine_per_plates():
    import xlrd
    import pandas as pd
    import numpy as np
    import re

    # Filter all warnings. If you would like to see the warnings, please comment the two lines below.
    import warnings
    warnings.filterwarnings('ignore')
    
    df = load_ticket_data()
    # YOUR CODE HERE
    
    # define a function to match license plate formats and return them
    def plate_format(s):
    
        if len(re.findall("^[a-zA-Z]{3}\d{4}$", s))!=0:
            return "ABC1234"
        elif len(re.findall("^[a-zA-Z]{3}\d{3}$", s))!=0:
            return "ABC123"
        elif len(re.findall("^\d{3}[a-zA-Z]{3}$", s))!=0:
            return "123ABC"
        else:
            return "vanity"
    
    # create a new column PlateFormat specifying the license plate format based on the Plate column
    df["PlateFormat"]=df.apply(lambda x: plate_format(str(x["Plate"])), axis=1)
    
    # return a dictionary of count of each license plate format for the state of MI
    plates_dict=dict(df.loc[df.State=="MI"].PlateFormat.value_counts())
    
    return plates_dict
    
#     raise NotImplementedError()

In [8]:
assert len(fine_per_plates())==4, "Return a dictionary with four items."
